## Main Driver Notebook for Training Graph NNs on Image Superpixels Datasets

### MODELS
- GatedGCN 
- GCN 
- GAT 
- GraphSage  
- GIN  
- MoNet  
- MLP  
- RingGNN  
- 3WLGNN    

### DATASET
- MNIST  
- CIFAR10

### TASK
- Graph Classification, i.e. Image Classification in classes

In [1]:
"""
    IMPORTING LIBS
"""
import dgl

import numpy as np
import os
import socket
import time
import random
import glob
import argparse, json
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

from tensorboardX import SummaryWriter
from tqdm import tqdm

import yaml

class DotDict(dict):
    def __init__(self, **kwds):
        self.update(kwds)
        self.__dict__ = self

Using backend: pytorch


In [2]:
# """
#     AUTORELOAD IPYTHON EXTENSION FOR RELOADING IMPORTED MODULES
# """

def in_ipynb():
    try:
        cfg = get_ipython().config 
        return True
    except NameError:
        return False
    
notebook_mode = in_ipynb()
print(notebook_mode)

if notebook_mode == True:
    %load_ext autoreload
    %autoreload 2

True


In [3]:
"""
    IMPORTING CUSTOM MODULES/METHODS
"""
from nets.superpixels_graph_classification.load_net import gnn_model # import all GNNS
from data.data import LoadData # import dataset


In [4]:
"""
    GPU Setup
"""
def gpu_setup(use_gpu, gpu_id):
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)  

    if torch.cuda.is_available() and use_gpu:
        print('cuda available with GPU:',torch.cuda.get_device_name(0))
        device = torch.device("cuda")
    else:
        print('cuda not available')
        device = torch.device("cpu")
    return device


# select GPU or CPU
#use_gpu = True; gpu_id = 0; device = None # default GPU
use_gpu = False; gpu_id = -1; device = None # CPU


In [5]:
# """
#     USER CONTROLS
# """
if notebook_mode == True:
    
    # MODEL_NAME = '3WLGNN'
    # MODEL_NAME = 'RingGNN'
    # MODEL_NAME = 'GatedGCN'
    # MODEL_NAME = 'MoNet'
    MODEL_NAME = 'GCN'
    # MODEL_NAME = 'GAT'
    # MODEL_NAME = 'GraphSage'
    # MODEL_NAME = 'DiffPool'
    # MODEL_NAME = 'MLP'
    # MODEL_NAME = 'GIN'

    DATASET_NAME = 'MNIST'
    # DATASET_NAME = 'CIFAR10'

    out_dir = 'out/superpixels_graph_classification/'
    root_log_dir = out_dir + 'logs/' + MODEL_NAME + "_" + DATASET_NAME + "_" + time.strftime('%Hh%Mm%Ss_on_%b_%d_%Y')
    root_ckpt_dir = out_dir + 'checkpoints/' + MODEL_NAME + "_" + DATASET_NAME + "_" + time.strftime('%Hh%Mm%Ss_on_%b_%d_%Y')

    print("[I] Loading data (notebook) ...")
    dataset = LoadData(DATASET_NAME)
    trainset, valset, testset = dataset.train, dataset.val, dataset.test
    print("[I] Finished loading.")

[I] Loading data (notebook) ...
[I] Loading dataset MNIST...
train, test, val sizes : 55000 10000 5000
[I] Finished loading.
[I] Data load time: 21.3327s
train dataset 0 tensor(5444)
train dataset 1 tensor(6179)
train dataset 2 tensor(5470)
train dataset 3 tensor(5638)
train dataset 4 tensor(5307)
train dataset 5 tensor(4987)
train dataset 6 tensor(5417)
train dataset 7 tensor(5715)
train dataset 8 tensor(5389)
train dataset 9 tensor(5454)
[I] Finished loading.


In [6]:
#MODEL_NAME = 'RingGNN'
#MODEL_NAME = 'GatedGCN'
MODEL_NAME = 'GCN'
#MODEL_NAME = 'GAT'
#MODEL_NAME = 'GraphSage'
#MODEL_NAME = 'MLP'
#MODEL_NAME = 'DiffPool'
#MODEL_NAME = 'GIN'
#MODEL_NAME = 'MoNet'


In [7]:
# """
#     PARAMETERS
# """
if notebook_mode == True:

    n_heads = -1
    edge_feat = False
    pseudo_dim_MoNet = -1
    kernel = -1
    gnn_per_block = -1
    embedding_dim = -1
    pool_ratio = -1
    n_mlp_GIN = -1
    gated = False
    self_loop = False
    #self_loop = True
    max_time = 12
    
    

    if MODEL_NAME == 'GatedGCN':
        seed=41; epochs=1000; batch_size=5; init_lr=5e-5; lr_reduce_factor=0.5; lr_schedule_patience=25; min_lr = 1e-6; weight_decay=0
        L=4; hidden_dim=70; out_dim=hidden_dim; dropout=0.0; readout='mean'

    if MODEL_NAME == 'GCN':
        seed=41; epochs=100; batch_size=128; init_lr=0.001; lr_reduce_factor=0.5; lr_schedule_patience=10; min_lr = 1e-5; weight_decay=0
        L=1; hidden_dim=146; out_dim=hidden_dim; dropout=0.0; readout='mean'

    if MODEL_NAME == 'GAT':
        seed=41; epochs=1000; batch_size=50; init_lr=5e-5; lr_reduce_factor=0.5; lr_schedule_patience=25; min_lr = 1e-6; weight_decay=0
        L=4; n_heads=8; hidden_dim=19; out_dim=n_heads*hidden_dim; dropout=0.0; readout='mean'
        print('True hidden dim:',out_dim)

    if MODEL_NAME == 'GraphSage':
        seed=41; epochs=1000; batch_size=50; init_lr=5e-5; lr_reduce_factor=0.5; lr_schedule_patience=25; min_lr = 1e-6; weight_decay=0
        L=4; hidden_dim=108; out_dim=hidden_dim; dropout=0.0; readout='mean'

    if MODEL_NAME == 'MLP':
        seed=41; epochs=1000; batch_size=50; init_lr=5e-4; lr_reduce_factor=0.5; lr_schedule_patience=25; min_lr = 1e-6; weight_decay=0
        gated=False; # MEAN
        L=4; hidden_dim=168; out_dim=hidden_dim; dropout=0.0; readout='mean'
        gated=True; # GATED
        L=4; hidden_dim=150; out_dim=hidden_dim; dropout=0.0; readout='mean'
        
    if MODEL_NAME == 'DiffPool':
        seed=41; epochs=1000; batch_size=50; init_lr=5e-4; lr_reduce_factor=0.5; lr_schedule_patience=25; min_lr = 1e-6; weight_decay=0
        L=4; hidden_dim=32; out_dim=hidden_dim; dropout=0.0; readout='mean'
        n_heads=8; gnn_per_block=3; batch_size=128; pool_ratio=0.15
        embedding_dim=32; # MNIST
        #embedding_dim=16; # CIFAR10

    if MODEL_NAME == 'GIN':
        seed=41; epochs=1000; batch_size=50; init_lr=5e-4; lr_reduce_factor=0.5; lr_schedule_patience=25; min_lr = 1e-6; weight_decay=0
        L=4; hidden_dim=110; out_dim=hidden_dim; dropout=0.0; readout='mean'
        n_mlp_GIN = 2; learn_eps_GIN=True; neighbor_aggr_GIN='sum'

    if MODEL_NAME == 'MoNet':
        seed=41; epochs=1000; batch_size=50; init_lr=5e-4; lr_reduce_factor=0.5; lr_schedule_patience=25; min_lr = 1e-6; weight_decay=0
        L=4; hidden_dim=90; out_dim=hidden_dim; dropout=0.0; readout='mean'
        pseudo_dim_MoNet=2; kernel=3;
        
    if MODEL_NAME == 'RingGNN':
        seed=41; epochs=1000; batch_size=1; init_lr=5e-5; lr_reduce_factor=0.5; lr_schedule_patience=25; min_lr = 1e-6; weight_decay=0
        #L=4; hidden_dim=145; out_dim=hidden_dim; dropout=0.0; readout='mean'
        L=4; hidden_dim=25; out_dim=hidden_dim; dropout=0.0;
        
    if MODEL_NAME == '3WLGNN':
        seed=41; epochs=1000; batch_size=1; init_lr=5e-5; lr_reduce_factor=0.5; lr_schedule_patience=25; min_lr = 1e-6; weight_decay=0
        #L=4; hidden_dim=145; out_dim=hidden_dim; dropout=0.0; readout='mean'
        L=3; hidden_dim=80; out_dim=hidden_dim; dropout=0.0;
      
    
    
    # generic new_params
    net_params = {}
    net_params['device'] = device
    net_params['gated'] = gated  # for mlpnet baseline
    net_params['in_dim'] = trainset[0][0].ndata['feat'][0].size(0)
    net_params['in_dim_edge'] = trainset[0][0].edata['feat'][0].size(0)
    net_params['residual'] = False
    net_params['hidden_dim'] = hidden_dim
    net_params['out_dim'] = out_dim
    num_classes = len(np.unique(np.array(trainset[:][1])))
    net_params['n_classes'] = num_classes
    net_params['n_heads'] = n_heads
    net_params['L'] = L  # min L should be 2
    net_params['readout'] = "sum"
    net_params['layer_norm'] = True
    net_params['batch_norm'] = True
    net_params['in_feat_dropout'] = 0.0
    net_params['dropout'] = 0.0
    net_params['edge_feat'] = edge_feat
    net_params['self_loop'] = self_loop

    # for MLPNet 
    net_params['gated'] = gated  
    
    # specific for MoNet
    net_params['pseudo_dim_MoNet'] = pseudo_dim_MoNet
    net_params['kernel'] = kernel
    
    # specific for GIN
    net_params['n_mlp_GIN'] = n_mlp_GIN
    net_params['learn_eps_GIN'] = True
    net_params['neighbor_aggr_GIN'] = 'sum'
    
    # specific for graphsage
    net_params['sage_aggregator'] = 'meanpool'    

    # specific for diffpoolnet
    net_params['data_mode'] = 'default'
    net_params['gnn_per_block'] = gnn_per_block
    net_params['embedding_dim'] = embedding_dim     
    net_params['pool_ratio'] = pool_ratio
    net_params['linkpred'] = True
    net_params['num_pool'] = 1
    net_params['cat'] = False
    net_params['batch_size'] = batch_size   

    # specific for RingGNN
    net_params['radius'] = 2
    num_nodes_train = [trainset[i][0].number_of_nodes() for i in range(len(trainset))]
    num_nodes_test = [testset[i][0].number_of_nodes() for i in range(len(testset))]
    num_nodes = num_nodes_train + num_nodes_test
    net_params['avg_node_num'] = int(np.ceil(np.mean(num_nodes)))
    
    # specific for 3WLGNN
    net_params['depth_of_mlp'] = 2
    
    # calculate assignment dimension: pool_ratio * largest graph's maximum
    # number of nodes  in the dataset
    max_num_nodes_train = max(num_nodes_train)
    max_num_nodes_test = max(num_nodes_test)
    max_num_node = max(max_num_nodes_train, max_num_nodes_test)
    net_params['assign_dim'] = int(max_num_node * net_params['pool_ratio']) * net_params['batch_size']


In [8]:
"""
    VIEWING MODEL CONFIG AND PARAMS
"""
def view_model_param(MODEL_NAME, net_params):
    model = gnn_model(MODEL_NAME, net_params)
    total_param = 0
    print("MODEL DETAILS:\n")
    #print(model)
    for param in model.parameters():
        # print(param.data.size())
        total_param += np.prod(list(param.data.size()))
    print('MODEL/Total parameters:', MODEL_NAME, total_param)
    return total_param


if notebook_mode == True:
    view_model_param(MODEL_NAME, net_params)

MODEL DETAILS:

MODEL/Total parameters: GCN 36103


In [9]:
"""
    TRAINING CODE
"""

def train_val_pipeline(MODEL_NAME, dataset, params, net_params, dirs):
    t0 = time.time()
    per_epoch_time = []
        
    DATASET_NAME = dataset.name
    
    if MODEL_NAME in ['GCN', 'GAT']:
        if net_params['self_loop']:
            print("[!] Adding graph self-loops for GCN/GAT models (central node trick).")
            dataset._add_self_loops()
    
    trainset, valset, testset = dataset.train, dataset.val, dataset.test
        
    root_log_dir, root_ckpt_dir, write_file_name, write_config_file, saved_model_name = dirs
    device = net_params['device']
    
    # Write the network and optimization hyper-parameters in folder config/

    # Delete not parsable config parameters
    save_params = net_params.copy()
    del save_params['device']
    del save_params['total_param']

    #model_name = saved_model_name
    data = {
        saved_model_name :{
            'model_path': saved_model_name + '/final.pkl',
            'Dataset': DATASET_NAME,
            'Model': MODEL_NAME,
            'params': params,
            'net_params': save_params,
            'total_param': (int)(net_params['total_param'])
        }
    }

    yaml_data = yaml.dump(data, default_flow_style=False)

    with open(write_config_file + '.yaml', 'w') as f:
        f.write(yaml_data)
        # f.write("""Dataset: {},\nModel: {}\n\nparams: {}\n\nnet_params: {}\n\n\nTotal Parameters: {}\n\n"""\
        #         .format(DATASET_NAME, MODEL_NAME, yaml.dump(params), yaml.dump(net_params), net_params['total_param']))

        
    # log_dir = os.path.join(root_log_dir, "RUN_" + str(0))
    log_dir = root_log_dir
    writer = SummaryWriter(log_dir=log_dir)

    # setting seeds
    random.seed(params['seed'])
    np.random.seed(params['seed'])
    torch.manual_seed(params['seed'])
    if device.type == 'cuda':
        torch.cuda.manual_seed(params['seed'])
    
    print("Training Graphs: ", len(trainset))
    print("Validation Graphs: ", len(valset))
    print("Test Graphs: ", len(testset))
    print("Number of Classes: ", net_params['n_classes'])

    model = gnn_model(MODEL_NAME, net_params)
    model = model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=params['init_lr'], weight_decay=params['weight_decay'])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                     factor=params['lr_reduce_factor'],
                                                     patience=params['lr_schedule_patience'],
                                                     verbose=True)
    
    epoch_train_losses, epoch_val_losses = [], []
    epoch_train_accs, epoch_val_accs = [], [] 
    
    # batching exception for Diffpool
    drop_last = True if MODEL_NAME == 'DiffPool' else False
    
    if MODEL_NAME in ['RingGNN', '3WLGNN']:
        # import train functions specific for WL-GNNs
        from train.train_superpixels_graph_classification import train_epoch_dense as train_epoch, evaluate_network_dense as evaluate_network

        train_loader = DataLoader(trainset, shuffle=True, collate_fn=dataset.collate_dense_gnn)
        val_loader = DataLoader(valset, shuffle=False, collate_fn=dataset.collate_dense_gnn)
        test_loader = DataLoader(testset, shuffle=False, collate_fn=dataset.collate_dense_gnn)

    else:
        # import train functions for all other GCNs
        from train.train_superpixels_graph_classification import train_epoch_sparse as train_epoch, evaluate_network_sparse as evaluate_network

        train_loader = DataLoader(trainset, batch_size=params['batch_size'], shuffle=True, drop_last=drop_last, collate_fn=dataset.collate)
        val_loader = DataLoader(valset, batch_size=params['batch_size'], shuffle=False, drop_last=drop_last, collate_fn=dataset.collate)
        test_loader = DataLoader(testset, batch_size=params['batch_size'], shuffle=False, drop_last=drop_last, collate_fn=dataset.collate)

    # At any point you can hit Ctrl + C to break out of training early.
    try:
        with tqdm(range(params['epochs'])) as t:
            for epoch in t:

                t.set_description('Epoch %d' % epoch)

                start = time.time()

                if MODEL_NAME in ['RingGNN', '3WLGNN']: # since different batch training function for dense GNNs
                    epoch_train_loss, epoch_train_acc, optimizer = train_epoch(model, optimizer, device, train_loader, epoch, params['batch_size'])
                else:   # for all other models common train function
                    epoch_train_loss, epoch_train_acc, optimizer = train_epoch(model, optimizer, device, train_loader, epoch)

                epoch_val_loss, epoch_val_acc = evaluate_network(model, device, val_loader, epoch)
                _, epoch_test_acc = evaluate_network(model, device, test_loader, epoch)                
                
                epoch_train_losses.append(epoch_train_loss)
                epoch_val_losses.append(epoch_val_loss)
                epoch_train_accs.append(epoch_train_acc)
                epoch_val_accs.append(epoch_val_acc)

                writer.add_scalar('train/_loss', epoch_train_loss, epoch)
                writer.add_scalar('val/_loss', epoch_val_loss, epoch)
                writer.add_scalar('train/_acc', epoch_train_acc, epoch)
                writer.add_scalar('val/_acc', epoch_val_acc, epoch)
                writer.add_scalar('test/_acc', epoch_test_acc, epoch)
                writer.add_scalar('learning_rate', optimizer.param_groups[0]['lr'], epoch)

                
                t.set_postfix(time=time.time()-start, lr=optimizer.param_groups[0]['lr'],
                              train_loss=epoch_train_loss, val_loss=epoch_val_loss,
                              train_acc=epoch_train_acc, val_acc=epoch_val_acc,
                              test_acc=epoch_test_acc)    

                per_epoch_time.append(time.time()-start)

                # Saving checkpoint
                ckpt_dir = os.path.join(root_ckpt_dir, "RUN")
                if not os.path.exists(ckpt_dir):
                    os.makedirs(ckpt_dir)
                torch.save(model.state_dict(), '{}.pkl'.format(ckpt_dir + "/epoch_" + str(epoch)))

                files = glob.glob(ckpt_dir + '/*.pkl')
                for file in files:
                    epoch_nb = file.split('_')[-1]
                    epoch_nb = int(epoch_nb.split('.')[0])
                    if epoch_nb < epoch-1:
                        os.remove(file)

                scheduler.step(epoch_val_loss)

                if optimizer.param_groups[0]['lr'] < params['min_lr']:
                    print("\n!! LR EQUAL TO MIN LR SET.")
                    break
                    
                # Stop training after params['max_time'] hours
                if time.time()-t0 > params['max_time']*3600:
                    print('-' * 89)
                    print("Max_time for training elapsed {:.2f} hours, so stopping".format(params['max_time']))
                    break
    
    except KeyboardInterrupt:
        print('-' * 89)
        print('Exiting from training early because of KeyboardInterrupt')

    # Saving final netwok
    torch.save(model.state_dict(), '{}.pkl'.format(root_ckpt_dir + "final"))
    
    _, test_acc = evaluate_network(model, device, test_loader, epoch)
    _, train_acc = evaluate_network(model, device, train_loader, epoch)
    print("Test Accuracy: {:.4f}".format(test_acc))
    print("Train Accuracy: {:.4f}".format(train_acc))
    print("Convergence Time (Epochs): {:.4f}".format(epoch))
    print("TOTAL TIME TAKEN: {:.4f}s".format(time.time()-t0))
    print("AVG TIME PER EPOCH: {:.4f}s".format(np.mean(per_epoch_time)))

    writer.close()

    """
        Write the results in out_dir/results folder
    """
    with open(write_file_name + '.txt', 'w') as f:
        f.write("""Dataset: {},\nModel: {}\n\nparams={}\n\nnet_params={}\n\n{}\n\nTotal Parameters: {}\n\n
    FINAL RESULTS\nTEST ACCURACY: {:.4f}\nTRAIN ACCURACY: {:.4f}\n\n
    Convergence Time (Epochs): {:.4f}\nTotal Time Taken: {:.4f} hrs\nAverage Time Per Epoch: {:.4f} s\n\n\n"""\
          .format(DATASET_NAME, MODEL_NAME, params, net_params, model, net_params['total_param'],
                  np.mean(np.array(test_acc))*100, np.mean(np.array(train_acc))*100, epoch, (time.time()-t0)/3600, np.mean(per_epoch_time)))
               

In [10]:
def main(notebook_mode=False,config=None):
    
    """
        USER CONTROLS
    """
    
    # terminal mode
    if notebook_mode==False:
        
        parser = argparse.ArgumentParser()
        parser.add_argument('--config', help="Please give a config.json file with training/model/data/param details")
        parser.add_argument('--gpu_id', help="Please give a value for gpu id")
        parser.add_argument('--model', help="Please give a value for model name")
        parser.add_argument('--dataset', help="Please give a value for dataset name")
        parser.add_argument('--out_dir', help="Please give a value for out_dir")
        parser.add_argument('--seed', help="Please give a value for seed")
        parser.add_argument('--epochs', help="Please give a value for epochs")
        parser.add_argument('--batch_size', help="Please give a value for batch_size")
        parser.add_argument('--init_lr', help="Please give a value for init_lr")
        parser.add_argument('--lr_reduce_factor', help="Please give a value for lr_reduce_factor")
        parser.add_argument('--lr_schedule_patience', help="Please give a value for lr_schedule_patience")
        parser.add_argument('--min_lr', help="Please give a value for min_lr")
        parser.add_argument('--weight_decay', help="Please give a value for weight_decay")
        parser.add_argument('--print_epoch_interval', help="Please give a value for print_epoch_interval")    
        parser.add_argument('--L', help="Please give a value for L")
        parser.add_argument('--hidden_dim', help="Please give a value for hidden_dim")
        parser.add_argument('--out_dim', help="Please give a value for out_dim")
        parser.add_argument('--residual', help="Please give a value for residual")
        parser.add_argument('--edge_feat', help="Please give a value for edge_feat")
        parser.add_argument('--readout', help="Please give a value for readout")
        parser.add_argument('--kernel', help="Please give a value for kernel")
        parser.add_argument('--n_heads', help="Please give a value for n_heads")
        parser.add_argument('--gated', help="Please give a value for gated")
        parser.add_argument('--in_feat_dropout', help="Please give a value for in_feat_dropout")
        parser.add_argument('--dropout', help="Please give a value for dropout")
        parser.add_argument('--layer_norm', help="Please give a value for layer_norm")
        parser.add_argument('--batch_norm', help="Please give a value for batch_norm")
        parser.add_argument('--sage_aggregator', help="Please give a value for sage_aggregator")
        parser.add_argument('--data_mode', help="Please give a value for data_mode")
        parser.add_argument('--num_pool', help="Please give a value for num_pool")
        parser.add_argument('--gnn_per_block', help="Please give a value for gnn_per_block")
        parser.add_argument('--embedding_dim', help="Please give a value for embedding_dim")
        parser.add_argument('--pool_ratio', help="Please give a value for pool_ratio")
        parser.add_argument('--linkpred', help="Please give a value for linkpred")
        parser.add_argument('--cat', help="Please give a value for cat")
        parser.add_argument('--self_loop', help="Please give a value for self_loop")
        parser.add_argument('--max_time', help="Please give a value for max_time")
        args = parser.parse_args()
        with open(args.config) as f:
            config = json.load(f)
            

        # device
        if args.gpu_id is not None:
            config['gpu']['id'] = int(args.gpu_id)
            config['gpu']['use'] = True
        device = gpu_setup(config['gpu']['use'], config['gpu']['id'])

        # model, dataset, out_dir
        if args.model is not None:
            MODEL_NAME = args.model
        else:
            MODEL_NAME = config['model']
        if args.dataset is not None:
            DATASET_NAME = args.dataset
        else:
            DATASET_NAME = config['dataset']
        dataset = LoadData(DATASET_NAME, portion_of_train=0.1)
        if args.out_dir is not None:
            out_dir = args.out_dir
        else:
            out_dir = config['out_dir']

        # parameters
        params = config['params']
        if args.seed is not None:
            params['seed'] = int(args.seed)
        if args.epochs is not None:
            params['epochs'] = int(args.epochs)
        if args.batch_size is not None:
            params['batch_size'] = int(args.batch_size)
        if args.init_lr is not None:
            params['init_lr'] = float(args.init_lr)
        if args.lr_reduce_factor is not None:
            params['lr_reduce_factor'] = float(args.lr_reduce_factor)
        if args.lr_schedule_patience is not None:
            params['lr_schedule_patience'] = int(args.lr_schedule_patience)
        if args.min_lr is not None:
            params['min_lr'] = float(args.min_lr)
        if args.weight_decay is not None:
            params['weight_decay'] = float(args.weight_decay)
        if args.print_epoch_interval is not None:
            params['print_epoch_interval'] = int(args.print_epoch_interval)
        if args.max_time is not None:
            params['max_time'] = float(args.max_time)

        # network parameters
        net_params = config['net_params']
        net_params['device'] = device
        net_params['gpu_id'] = config['gpu']['id']
        net_params['batch_size'] = params['batch_size']
        if args.L is not None:
            net_params['L'] = int(args.L)
        if args.hidden_dim is not None:
            net_params['hidden_dim'] = int(args.hidden_dim)
        if args.out_dim is not None:
            net_params['out_dim'] = int(args.out_dim)   
        if args.residual is not None:
            net_params['residual'] = True if args.residual=='True' else False
        if args.edge_feat is not None:
            net_params['edge_feat'] = True if args.edge_feat=='True' else False
        if args.readout is not None:
            net_params['readout'] = args.readout
        if args.kernel is not None:
            net_params['kernel'] = int(args.kernel)
        if args.n_heads is not None:
            net_params['n_heads'] = int(args.n_heads)
        if args.gated is not None:
            net_params['gated'] = True if args.gated=='True' else False
        if args.in_feat_dropout is not None:
            net_params['in_feat_dropout'] = float(args.in_feat_dropout)
        if args.dropout is not None:
            net_params['dropout'] = float(args.dropout)
        if args.layer_norm is not None:
            net_params['layer_norm'] = True if args.layer_norm=='True' else False
        if args.batch_norm is not None:
            net_params['batch_norm'] = True if args.batch_norm=='True' else False
        if args.sage_aggregator is not None:
            net_params['sage_aggregator'] = args.sage_aggregator
        if args.data_mode is not None:
            net_params['data_mode'] = args.data_mode
        if args.num_pool is not None:
            net_params['num_pool'] = int(args.num_pool)
        if args.gnn_per_block is not None:
            net_params['gnn_per_block'] = int(args.gnn_per_block)
        if args.embedding_dim is not None:
            net_params['embedding_dim'] = int(args.embedding_dim)
        if args.pool_ratio is not None:
            net_params['pool_ratio'] = float(args.pool_ratio)
        if args.linkpred is not None:
            net_params['linkpred'] = True if args.linkpred=='True' else False
        if args.cat is not None:
            net_params['cat'] = True if args.cat=='True' else False
        if args.self_loop is not None:
            net_params['self_loop'] = True if args.self_loop=='True' else False

            
    # notebook mode
    if notebook_mode:
        
        # parameters
        params = config['params']
        
        # dataset
        DATASET_NAME = config['dataset']
        dataset = LoadData(DATASET_NAME)
        
        # device
        device = gpu_setup(config['gpu']['use'], config['gpu']['id'])
        out_dir = config['out_dir']
        
        # GNN model
        MODEL_NAME = config['model']
        
        # network parameters
        net_params = config['net_params']
        net_params['device'] = device
        net_params['gpu_id'] = config['gpu']['id']
        net_params['batch_size'] = params['batch_size']
              
            
    # Superpixels
    net_params['in_dim'] = dataset.train[0][0].ndata['feat'][0].size(0)
    net_params['in_dim_edge'] = dataset.train[0][0].edata['feat'][0].size(0)
    num_classes = len(np.unique(np.array(dataset.train[:][1])))
    net_params['n_classes'] = num_classes

    if MODEL_NAME == 'DiffPool':
        # calculate assignment dimension: pool_ratio * largest graph's maximum
        # number of nodes  in the dataset
        max_num_nodes_train = max([dataset.train[i][0].number_of_nodes() for i in range(len(dataset.train))])
        max_num_nodes_test = max([dataset.test[i][0].number_of_nodes() for i in range(len(dataset.test))])
        max_num_node = max(max_num_nodes_train, max_num_nodes_test)
        net_params['assign_dim'] = int(max_num_node * net_params['pool_ratio']) * net_params['batch_size']
        
    if MODEL_NAME == 'RingGNN':
        num_nodes_train = [dataset.train[i][0].number_of_nodes() for i in range(len(dataset.train))]
        num_nodes_test = [dataset.test[i][0].number_of_nodes() for i in range(len(dataset.test))]
        num_nodes = num_nodes_train + num_nodes_test
        net_params['avg_node_num'] = int(np.ceil(np.mean(num_nodes)))
        
    
    #overwirting out variabel
    out_dir = "../models/individual_models/"

    saved_model_name = MODEL_NAME + "_" + DATASET_NAME + "_GPU" + str(config['gpu']['id']) + "_" + time.strftime('%Hh%Mm%Ss_on_%b_%d_%Y')
    saveFolder = saved_model_name + "/"

    root_log_dir = out_dir + saveFolder + 'log'
    root_ckpt_dir = out_dir + saveFolder
    write_file_name = out_dir + saveFolder + 'result'

    write_config_folder = "../src/conf/models/individual_models/"

    write_config_file = write_config_folder + saved_model_name
    dirs = root_log_dir, root_ckpt_dir, write_file_name, write_config_file, saved_model_name

    if not os.path.exists(out_dir + saveFolder):
        os.makedirs(out_dir + saveFolder)
    else:
        print("\nFolder already exists. Contents might be overwritten\n")

    # if not os.path.exists(out_dir + 'configs'):
    #     os.makedirs(out_dir + 'configs')

    net_params['total_param'] = view_model_param(MODEL_NAME, net_params)
    train_val_pipeline(MODEL_NAME, dataset, params, net_params, dirs)

    
    
if notebook_mode==True:
    
    config = {}
    # gpu config
    gpu = {}
    gpu['use'] = use_gpu
    gpu['id'] = gpu_id
    config['gpu'] = gpu
    # GNN model, dataset, out_dir
    config['model'] = MODEL_NAME
    config['dataset'] = DATASET_NAME
    config['out_dir'] = out_dir
    # parameters
    params = {}
    params['seed'] = seed
    params['epochs'] = epochs
    params['batch_size'] = batch_size
    params['init_lr'] = init_lr
    params['lr_reduce_factor'] = lr_reduce_factor 
    params['lr_schedule_patience'] = lr_schedule_patience
    params['min_lr'] = min_lr
    params['weight_decay'] = weight_decay
    params['print_epoch_interval'] = 5
    params['max_time'] = max_time
    config['params'] = params
    # network parameters
    config['net_params'] = net_params
    
    # convert to .py format
    from utils.cleaner_main import *
    cleaner_main('main_superpixels_graph_classification')
    
    main(True,config)
    
else:
    
    main()
    

Convert main_superpixels_graph_classification.ipynb to main_superpixels_graph_classification.py


[NbConvertApp] Converting notebook main_superpixels_graph_classification.ipynb to script
[NbConvertApp] Writing 29523 bytes to main_superpixels_graph_classification.py


Clean main_superpixels_graph_classification.py
Done. 
[I] Loading dataset MNIST...
train, test, val sizes : 55000 10000 5000
[I] Finished loading.
[I] Data load time: 24.8461s
train dataset 0 tensor(5444)
train dataset 1 tensor(6179)
train dataset 2 tensor(5470)
train dataset 3 tensor(5638)
train dataset 4 tensor(5307)
train dataset 5 tensor(4987)
train dataset 6 tensor(5417)
train dataset 7 tensor(5715)
train dataset 8 tensor(5389)
train dataset 9 tensor(5454)
cuda not available
MODEL DETAILS:

MODEL/Total parameters: GCN 36103
Training Graphs:  55000
Validation Graphs:  5000
Test Graphs:  10000
Number of Classes:  10


/home/weronika/Documents/masters/sem3/deep_learning/gnn_fusion/.venv/lib/python3.8/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
Epoch 35:  35%|███▌      | 35/100 [07:05<13:21, 12.34s/it, lr=0.001, test_acc=0.669, time=12.3, train_acc=0.685, train_loss=0.918, val_acc=0.665, val_loss=0.946]

Epoch 00035: reducing learning rate of group 0 to 5.0000e-04.


...

Epoch 00048: reducing learning rate of group 0 to 2.5000e-04.


Epoch 96:  96%|█████████▌| 96/100 [18:37<00:43, 10.94s/it, lr=0.00025, test_acc=0.765, time=10.7, train_acc=0.759, train_loss=0.71, val_acc=0.763, val_loss=0.702] 

Epoch 00096: reducing learning rate of group 0 to 1.2500e-04.


Epoch 99: 100%|██████████| 100/100 [19:24<00:00, 11.64s/it, lr=0.000125, test_acc=0.769, time=12.7, train_acc=0.768, train_loss=0.683, val_acc=0.764, val_loss=0.682]


Test Accuracy: 0.7695
Train Accuracy: 0.7618
Convergence Time (Epochs): 99.0000
TOTAL TIME TAKEN: 1171.7001s
AVG TIME PER EPOCH: 11.6404s
